## Librerias

In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


## Conectar a AmigoCloud

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [3]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

## Datos de crédito

In [5]:
semana = 'S15'

In [6]:
query = {'query': 'select insp.canhero, prop.propiedad, lote.lote, gal.s3_filename, gal.amigocloud_url foto\
         from dataset_307560 insp\
         inner join dataset_307562 prop on insp.amigo_id=prop.insp_ref_prop\
         inner join dataset_307563 lote on prop.amigo_id=lote.prop_ref_lote\
         inner join gallery_46845 gal on lote.amigo_id=gal.source_amigo_id\
         where semana_1ra = \'{sem}\' order by canhero'.format(sem=semana)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
data = select['data']
#credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())
print('end')

end


In [7]:
data

[{'canhero': '2220 / CRUZ VARGAS TITO JHONNY ',
  'propiedad': '416 / LAS TRANQUERAS--OMAR CRUZ',
  'lote': 'L2',
  's3_filename': 'WhatsApp Image 2023-04-15 at 11.26.15 (1).jpeg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/bd75686fafe34c1da4d30fe676a45a24/15bff8cb3cb041c58dc2e438673f593a/WhatsApp%20Image%202023-04-15%20at%2011.26.15%20(1).jpeg'},
 {'canhero': '2220 / CRUZ VARGAS TITO JHONNY ',
  'propiedad': '201 / LA HONDONADA--CRUZ',
  'lote': 'L13.5',
  's3_filename': 'WhatsApp Image 2023-04-15 at 11.26.15.jpeg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/b4cc2ccaf3b8409b99334a63434953fe/bcc70a285326444c8dcfd4b9e6dfcb1e/WhatsApp%20Image%202023-04-15%20at%2011.26.15.jpeg'},
 {'canhero': '2510 / CRUZ BARRIENTOS OSCAR ANDREZ',
  'propiedad': '419 / PIRAICITO--CRUZ',
  'lote': 'L3.1',
  's3_filename': 'WhatsApp Image 2023-04-15 at 11.26.16.jpeg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/599a3287c2684e8fa8e2609a1b73a314/da

In [8]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate(ruta + "/templates/tpl_1ra_fotos.docx")

#descargar fotos y generar lista InlineImage
lista_fotos_inline = []
for foto in data:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta + '/fotos/' + foto['s3_filename'], width=Mm(120)), 'canhero': foto['canhero']})
    print(foto['s3_filename'])

context = {'fotos':lista_fotos_inline}

doc.render(context)
file_name = 'SEMANA - ' + semana
doc.save(ruta + '/_' + file_name + '.docx')

WhatsApp Image 2023-04-15 at 11.26.15 (1).jpeg
WhatsApp Image 2023-04-15 at 11.26.15.jpeg
WhatsApp Image 2023-04-15 at 11.26.16.jpeg
WhatsApp Image 2023-04-15 at 11.26.15 (2).jpeg
WhatsApp Image 2023-04-13 at 14.29.27.jpeg
WhatsApp Image 2023-04-15 at 11.07.55.jpeg
WhatsApp Image 2023-04-15 at 11.21.05.jpeg
WhatsApp Image 2023-04-15 at 11.21.05 (1).jpeg
